In [1]:
do_posts = True
overwrite = True

In [2]:
import sys 
sys.path.append('..')
import logging as lg
import re
import numpy as np
import coloredlogs
import constants as CS
import pandas as pd
from helpers import *
from tqdm import tqdm
import helpers.globals_loader as globals_loader
import helpers.helper_functions as helper_functions
coloredlogs.install()

CS.USE_MINIFIED_DATA = True
CS.LOAD_POSTS = True
CS.LOAD_COMMENTS = False
CS.LOAD_FOUNDATIONS = False
CS.LOAD_LIWC = False
globals_loader.init()

2021-11-12 00:41:40 PHDesktop root[1422] WARNING Using minified data (fraction: 0.05)
2021-11-12 00:41:40 PHDesktop root[1422] INFO Loading posts: /home/philipp/Document/Coding/Reddit_Morality_Analysis/datasets/data/posts_cleaned_27_6_2021_mini.csv
2021-11-12 00:41:41 PHDesktop root[1422] INFO Nr. of posts 30946
2021-11-12 00:41:41 PHDesktop root[1422] INFO Loading comments: /home/philipp/Document/Coding/Reddit_Morality_Analysis/datasets/data/comments_cleaned_16_07_2021_mini.csv
2021-11-12 00:41:41 PHDesktop root[1422] WARNING Skipping loading liwc...
2021-11-12 00:41:41 PHDesktop root[1422] WARNING Skipping loading foundations...
2021-11-12 00:41:41 PHDesktop root[1422] INFO Loading spacy


In [3]:
df = globals_loader.df_posts if do_posts else globals_loader.df_comments

In [4]:
prefix = "post" if do_posts else "comment"
df = globals_loader.df_posts if do_posts else globals_loader.df_comments
ids = df[prefix+"_id"].tolist()

reddit = globals_loader.reddit
elem_prefix = "t3" if do_posts else "t1"
ids = [i if i.startswith(elem_prefix+"_") else f"{elem_prefix}_{i}" for i in ids]
scores = []

post_idx = 0
comment_idx = 0


if do_posts:

    scores = np.zeros((len(ids),3))
    scores[:,0] = range(len(ids))
    scores = scores.tolist()
    
    scores_lst = []
    
    for submission in tqdm(reddit.info(ids), total=len(ids)):
        #submission = reddit.info(ids[i])
        updated = [ids[post_idx],0,0,""] #id, score, ratio
        updated[1] = submission.score
        updated[2] = submission.upvote_ratio
        updated[3] = submission.link_flair_text
        scores[post_idx] = updated
        
        scores_lst.append(updated)
        post_idx += 1
        
        # see https://api.reddit.com/api/info/?id=t3_apcnyn
else:
    
    scores = np.zeros((len(ids),2))
    scores[:,0] = range(len(ids))
    scores = scores.tolist()
    
    scores_lst = []

    for comment in tqdm(reddit.info(ids), total=len(ids)):
        #comment = reddit.info(ids[i])   

        updated = [comment_idx,0] #id, score, ratio
        updated[1] = comment.score
        scores[comment_idx] = updated
        scores_lst.append(updated)
        comment_idx+=1
        
        
        # comments do not have an upvote ratio
        # see https://api.reddit.com/api/info/?id=t1_eg7dfxx

# iterate over ids and check which ids are not in updated => for those we set the scores to None
# we check updated but modify updated_new
print("Original scores lst size {0}, ids size {1} => need to add {2} entries".format(len(scores_lst), len(ids), abs(len(scores_lst)-len(ids))))

100%|█████████▉| 30945/30946 [10:11<00:00, 50.61it/s]

Original scores lst size 30945, ids size 30946 => need to add 1 entries


In [7]:
    insertion_idxs = []
    if len(scores_lst) != len(ids):
        for i in range(len(ids)):
            if i < len(scores_lst):
                id_orig = ids[i]
                id_updated = scores_lst[i][0]
                if id_orig != id_updated:
                    insertion_idxs.append(i)
                    #empty_insert = [None] * len(scores_lst[i])
                    #empty_insert[0] = ids[i]
                    #scores_lst_new.insert(i, empty_insert)
            else:
                insertion_idxs.append(i)
        
        scores_lst_new = scores_lst.copy()
        for i in insertion_idxs:
            empty_insert = [None] * len(scores_lst[0])
            empty_insert[0] = ids[i]
            scores_lst_new.insert(i+insertion_idxs.index(i), empty_insert)

In [8]:
print(len(scores_lst_new) == len(ids))

True
